In [17]:
import git
import pickle
from cottools.scc import collect_scc, SccData
from cottools.filesolver import NameRepo
# This import will be slow... Needs to be fixed.
from concernbert.frontend import CdCalculator
import csv
from datetime import datetime

In [18]:
def load_bytes(repo: git.Repo, hexsha: str) -> bytes:
    return repo.odb.stream(bytes.fromhex(hexsha)).read()  # type: ignore

In [19]:
scc_data = collect_scc("_repos/activemq")

# with open("_pickles/activemq_scc.pkl", "wb") as f:
#     pickle.dump(scc_data, f)

# with open("_pickles/activemq_scc.pkl", "rb") as f:
#     scc_data: dict[str, SccData] = pickle.load(f)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 68806/68806

In [20]:
#file_repo = NameRepo.parse_log("_repos/activemq", "HEAD")
# file_repo.dump("_pickles/activemq.pkl")
file_repo = NameRepo.load("_pickles/activemq.pkl")

In [21]:
# name = "activemq-client/src/main/java/org/apache/activemq/ActiveMQConnection.java"
# name = "activemq-client/src/main/java/org/apache/activemq/ActiveMQMessageConsumer.java"
name = "activemq-client/src/main/java/org/apache/activemq/transport/failover/FailoverTransport.java"
file_id = file_repo.file_id_by_name(file_repo.latest_commit(), name)

print(len(file_repo.changes_by_name(name)))
print(len(file_repo.changes_by_id(file_id)))

38
163


In [22]:
changes = file_repo.changes_by_id(file_id)
len(changes)
cont_changes = file_repo.cont_changes_by_id(file_id)
len(cont_changes)

162

In [23]:
repo = git.Repo("_repos/activemq", odbt=git.GitCmdObjectDB)

In [24]:
cd_calculator = CdCalculator("_models/EntityBERT-v3_train_nonldl-lr5e5-2_83-e3", "_cache")

In [ ]:
with open(f"activemq-{file_id}.csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
# Write CSV header
        writer.writerow(['commit', 'commit_date', 'author_date', 'path', 'loc', 'lloc', 'comments', 'blanks', 'entities', 'inter_cd', 'intra_cd', 'cc'])
        for rev in file_repo.cont_changes_by_id(file_id):
            file_name = file_repo.file_name_by_id(rev, file_id)
            commit = repo.commit(rev)
            obj = commit.tree.join(file_name)
            if not isinstance(obj, git.Blob):
                raise RuntimeError
            content = load_bytes(repo,obj.hexsha)
            source = content.decode()
            cd = cd_calculator.calc_cd(source, pbar=False)
            print(cd.inter_cd)
            # print("Im HERE ")
            # print("Im HERE ")
            # print("Im HERE ")
            # print(source)
            #cd_calculator.calc_cd(source, pbar=True)
            loc = scc_data[obj.hexsha].loc
            blanks = scc_data[obj.hexsha].blanks
            complexity = scc_data[obj.hexsha].complexity
            # Now you can integrate this data into your CSV writing process:
            # Now integrate this data into your CSV writing process:
            writer.writerow([
                commit.hexsha,  # Commit Hash
                commit.committed_datetime,
                commit.authored_datetime,
                file_name,  # File Path
                loc,  # LOC (Lines of Code)
                loc,  # LLOC (Logical Lines of Code)
                commit.message.strip(),  # Comments
                blanks,  # Blanks 
                cd.inter_cd,  # Inter CD (from CD calculation)
                cd.intra_cd,  # Intra CD (from CD calculation)
                complexity  # Complexity (CC - Cyclomatic Complexity)
            ])

1.1965821361022442
1.1965648326329785
1.1981610031317613
1.1981610031317613
1.1981610031317613
1.1981610031317613
1.4244272138061094
1.4169559369541131
1.4139125385737124
1.4140588399641154
1.4147133322186742
1.4145100400182824
1.414046973053913
1.415160796637347
1.4150251368511644
1.414823143676729
1.4145459456767533
1.4121234706549575
1.4098508329202004
1.4160221198681953
1.4163610718928883
1.4166578890199513
1.35906298877772
1.3585260100367946
1.3584922198146556
1.3584494055421312
1.3591755046567067
1.3592264427492333
1.3592264427492333
1.3590699621285236
1.3154233669965791
1.3183110956484922
1.3137782990139923
1.3183110956484922
1.3120946488718315
1.322585375457223
1.322585375457223
1.3225896978492613
1.3174090544298591
1.3180920153195939
1.3180920153195939
1.335306234518368
1.335306234518368
1.3340152183111784
1.3353674370590438
1.3354932589583104
1.3346339369429852
1.333892381717487
1.346269777121638
1.3464006853609465
1.3487328026601721
1.3439283804285092
1.347077531333274
1.347

KeyboardInterrupt: 